<a href="https://colab.research.google.com/github/ronenfeldman/Practicum23/blob/main/yehonatan_cohen/using_openai_5_url.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip install

In [ ]:
# !pip install langchain
# !pip install openai
!pip install --upgrade transformers
!pip install --upgrade pillow
!pip install sentence_transformers
!pip install faiss-cpu
!pip install --upgrade pymupdf
!pip install PyPDF2
!pip install pdfplumber

In [ ]:
import fitz
import requests
import io
import pandas as pd
import os
import PyPDF2
import pdfplumber

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
file_path = r'/content/gdrive/MyDrive/MBA/Data Science Practicum 23/final_project/PDF files.txt'
file = open(file_path, 'r')

# Read the contents of the file
pdfs = file.read()

lines_pdfs = pdfs.split('\n')

In [ ]:
import aiohttp
import asyncio
import fitz
import pandas as pd

num_of_doc = 5
concurrent_tasks = 10  # Limiting to 10 concurrent tasks

async def worker(session, url, semaphore):
    async with semaphore:
        url = url.replace("\r", "")
        async with session.get(url) as response:
            file_data = await response.read()

        try:
            doc = fitz.open(stream=file_data, filetype='pdf')
        except Exception as e:
            print(f"Failed to open PDF from {url}. Error: {e}")
            return None

        text = " ".join([page.get_text() for page in doc])
        total_text_len = len(text)

        return {
            "pdf_name": url.split('/')[-1],
            "url": url,
            "page_count": doc.page_count,
            "text_len": total_text_len,
            "table_of_content": doc.get_toc(),
            "pdf_text": text
        }

async def fetch_pdfs(urls):
    async with aiohttp.ClientSession() as session:
        semaphore = asyncio.Semaphore(concurrent_tasks)
        tasks = [worker(session, url, semaphore) for url in urls[:num_of_doc]]
        return await asyncio.gather(*tasks)

%autoawait True

# Assuming lines_pdfs is a list of URLs
data = await fetch_pdfs(lines_pdfs)
df = pd.DataFrame([d for d in data if d is not None])  # Filtering out None values

In [ ]:
df.sample(5)

,pdf_name,url,page_count,text_len,table_of_content,pdf_text
2,NASDAQ_TWOU_2018.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,5,10889,[],"\n \n \n \n \n2U, Inc. is a diverse collectio..."
3,2019-2U-ESG-Report.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,10,19379,[],"\n \nEdtech with a human touch. \n2U, INC. ..."
4,NASDAQ_TWOU_2020.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,38,42271,[],"2020 ESG Report\nCulture, Equity, Education, H..."
0,Corporate-Governance-Report-Q2FY2013.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,1,1549,[],\n \n \n \nQUARTERLY COMPLIANCE REPORT ON COR...
1,Business-Responsibility-Report-FY-2017-18.pdf,https://sust-reports-prod-v1.s3.us-east-2.amaz...,6,23727,[],47\nAnnual Report 2017-18\nBusiness Responsibi...


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os
import openai

In [ ]:
OPENAI_API_KEY = "-----"

In [ ]:
llm = ChatOpenAI(temperature=0.9, openai_api_key=OPENAI_API_KEY)

In [ ]:
prompt = ChatPromptTemplate.from_template(
   """For the follow sentence please classify it into ONLY one of the following 5 categories based on its ESG related content. Give only the category name.

                                            -'Quantitative Evidence': The sentence contains quantitative evidence that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count!
                                            -'Action Evidence': The sentence contains hard evidence that is not quantitative that the company already has been taking actions to improve its ESG measures and already implemented the plans. Future plans do NOT count! .
                                            -'Suggestive Evidence': The sentence doesn't contain hard evidence of taking actions to improve the company's ESG measures, but it suggests that such actions have been taken already and have been implemented. Future plans do NOT count!
                                            -'Intention': The sentence mentions a company's plans, intentions or commitments of taking actions in the future, without evidence of any actual actions yet. It include just future plans of the company to take actions related to ESG.
                                            -'Other': The sentence does not match any category.

                                            sentence : {txt}"""
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

sample

In [ ]:
txt = "Our PCI target for 2028 is 71 g CO2e/MJ, a > 5% decrease from 2016."
chain.run(txt)

'Quantitative Evidence'

In [ ]:
txt = "The collaboration could lead to capturing and storing up to 50 million metric tons of CO2 per year by 2030 and about 100 million metric tons by 2040"
chain.run(txt)

'Quantitative Evidence'

In [ ]:
txt = "The organizations are considering using CCS technology at facilities that generate electricity and manufacture everyday products such as plastics, motor fuels and packaging."
chain.run(txt)

'Intention'

In [ ]:
def chunk_text(text, chunk_size):
    # Split the text by paragraph and chunk it
    chunks = text.split('\n')
    return [chunks[i:i + chunk_size] for i in range(0, len(chunks), chunk_size)]

In [ ]:
def class_row_in_text(num_doc, chunk_size= 10) :

      chunked_docs = chunk_text(df["pdf_text"][num_doc], chunk_size)
      chunked_docs_combined = ['\n'.join(chunked_docs) for chunked_docs in chunked_docs]

      responses =[]
      for row in chunked_docs_combined :
        response = chain.run(row)
        responses.append([num_doc, response, row])

      return responses

In [ ]:
all_responses = []
for i in df.index :
    responses = class_row_in_text(i)
    all_responses = all_responses + responses

In [ ]:
# Convert the list of lists to a DataFrame
all_responses_df = pd.DataFrame(all_responses, columns=['num_doc', 'response', 'row'])

all_responses_df

,num_doc,response,row
0,0,Other,\n \n \n \nQUARTERLY COMPLIANCE REPORT ON COR...
1,0,Other,Clause of \nListing \nAgreement \nCompliance \...
2,0,Other,4 \nI. Board of Directors \n49 (I) \n \n \n(A)...
3,0,Quantitative Evidence,Directors’ \ncompensation & disclosures \n49 ...
4,0,Other,(D) Code of Conduct \n49 (ID) \nYes \n \nII. A...
...,...,...,...
305,4,Action Evidence,2U’s Chief Privacy Oﬃcer leads our data privac...
306,4,Other,"and focuses on transparency, data collection a..."
307,4,Intention,the Private Securities Litigation Reform Act o...
308,4,Other,diﬀer materially from those contained in any f...


In [ ]:
all_responses_df["response"].value_counts()

Other                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [ ]:
def extract_category_name(category):
    category = category.lower()
    categories = ['quantitative evidence', 'action evidence', 'suggestive evidence', 'intention','other']
    for cat in categories:
        if cat in category:
            return cat.title()
    return ''

In [ ]:
extract_category_name('Category: suggestive evidence')

'Suggestive Evidence'

In [ ]:
all_responses_df["response_0"] = all_responses_df["response"].apply(extract_category_name)

In [ ]:
all_responses_df["response_0"].value_counts()

Other                    169
Quantitative Evidence     53
Action Evidence           49
Suggestive Evidence       24
Intention                 12
                           3
Name: response_0, dtype: int64

In [ ]:
all_responses_df.to_csv(r"/content/gdrive/MyDrive/MBA/Data Science Practicum 23/final_project/all_responses_df_openai_5.csv", index=False, escapechar='\\')